In [43]:
#from Recommenders.Hybrid.SimilarityMergedHybridRecommender import SimilarityMergedHybridRecommender
#from src.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaCBFRecommender import RP3betaCBFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from Recommenders.Hybrid.GeneralizedMergedHybridRecommender import GeneralizedMergedHybridRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_ElasticNet
from Recommenders.Implicit.FeatureCombinedImplicitALSRecommender import FeatureCombinedImplicitALSRecommender
from Recommenders.Hybrid.GeneralizedSimilarityMergedHybridRecommender import GeneralizedSimilarityMergedHybridRecommender
from Utils.ICM_preprocessing import *
from Utils.data_loader import load_URM, load_ICM
from Utils.confidence_scaling import *
from Utils.write_submission import write_submission
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.K_Fold_Evaluator import K_Fold_Evaluator_MAP
from bayes_opt import BayesianOptimization

In [44]:
URM_all = load_URM("input/data_train.csv")
ICM_all = load_ICM("input/data_ICM_subgenre.csv")

#ICM_combined = combine(ICM=ICM_all, URM = URM_all)

In [45]:
URMs_train = []
URMs_validation = []

for k in range(2):
    URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.80)
    URMs_train.append(URM_train)
    URMs_validation.append(URM_validation)

In [46]:
evaluator_validation = K_Fold_Evaluator_MAP(URMs_validation, cutoff_list=[10], verbose=False)

ICMs_combined = []
for URM in URMs_train:
    ICMs_combined.append(combine(ICM=ICM_all, URM=URM))

In [47]:
IALS_recommenders = []
rp3betaCBF_recommenders = []
userRP3beta_recommenders = []

for index in range(len(URMs_train)):
    IALS_recommenders.append(
        FeatureCombinedImplicitALSRecommender(
            URM_train=URMs_train[index],
            ICM_train=ICM_all,
            verbose=True
        )
    )
    IALS_recommenders[index].fit(
        factors=int(250),
        regularization=0.01,
        use_gpu=False,
        iterations=43,
        num_threads=4,
        confidence_scaling=linear_scaling_confidence,
        **{
            'URM': {"alpha": 50},
            'ICM': {"alpha": 50}
        }
    )

    rp3betaCBF_recommenders.append(
        RP3betaCBFRecommender(
            URM_train=URMs_train[index],
            ICM_train=ICMs_combined[index],
            verbose=False
        )
    )

    rp3betaCBF_recommenders[index].fit(
        topK=int(529.1628484087545),
        alpha=0.45304737831676245,
        beta=0.226647894170121,
        implicit=False
    )

    # userRP3beta_recommenders.append(
    #     UserRP3betaRecommender(
    #         URM_train=ICMs_combined[index].T,
    #         verbose=False
    #     )
    # )

    # userRP3beta_recommenders[index].fit(
    #     topK=int(252.2),
    #     alpha=0.2658,
    #     beta=0.2847,
    #     implicit=False
    # )

tuning_params = {
    "hybridWeight1": (0, 1),
    "hybridWeight2": (0, 1)
    #"hybridWeight3": (0, 1)
}

results = []


def BO_func(
        hybridWeight1,
        hybridWeight2
        #hybridWeight3
):
    recommenders = []

    for index in range(len(URMs_train)):

        recommender = GeneralizedMergedHybridRecommender(
            URM_train=URMs_train[index],
            recommenders=[
                IALS_recommenders[index],
                rp3betaCBF_recommenders[index],
                userRP3beta_recommenders[index]
            ],
            verbose=False
        )

        totalWeight = hybridWeight1+hybridWeight2#+hybridWeight3
        recommender.fit(
            alphas=[
                hybridWeight1/totalWeight,
                hybridWeight2/totalWeight
                #hybridWeight3/totalWeight
            ]
        )

        recommenders.append(recommender)

    result = evaluator_validation.evaluateRecommender(recommenders)
    results.append(result)
    return sum(result) / len(result)


optimizer = BayesianOptimization(
    f=BO_func,
    pbounds=tuning_params,
    verbose=5,
    random_state=5,
)

optimizer.maximize(
    init_points=30,
    n_iter=50,
)

recommender = GeneralizedMergedHybridRecommender(
            URM_train=URMs_train[0],
            recommenders=[
                IALS_recommenders[0],
                rp3betaCBF_recommenders[0]
                #userRP3beta_recommenders[0]
            ],
            verbose=False
        )
recommender.fit()

100%|██████████| 43/43 [04:18<00:00,  6.00s/it]


|   iter    |  target   | hybrid... | hybrid... |
-------------------------------------------------


IndexError: list index out of range

In [ ]:

# p3alpha_recommender = P3alphaRecommender(
#     URM_train=URM_all,
#     verbose=False
# )
#
# p3alpha_recommender.fit(
#     topK=int(212.8832860130684),
#     alpha=0.4729294763382114,
#     implicit=True
# )


In [ ]:
# RP3 BETA (Combined) -----------------------------------------

# rp3betaCombined_recommender= RP3betaCBFRecommender(
#     URM_train=URM_all,
#     ICM_train=ICM_combined,
#     verbose=False
# )

# rp3betaCombined_recommender.fit(
#     topK=int(529.1628484087545),
#     alpha=0.45304737831676245,
#     beta=0.226647894170121,
#     implicit=True
# )

In [ ]:
%env OPENBLAS_NUM_THREADS=1

env: OPENBLAS_NUM_THREADS=1


In [ ]:
# IALS -----------------------------------------

# IALS_recommender= FeatureCombinedImplicitALSRecommender(
#     URM_train=URM_all,
#     ICM_train=ICM_all,
#     verbose=True
# )

# IALS_recommender.fit(
#     factors=int(398.601583855084),
#     regularization=0.01,
#     use_gpu=False,
#     iterations=int(94.22855449116447),
#     num_threads=6,
#     confidence_scaling=linear_scaling_confidence,
#     **{
#         'URM': {"alpha": 42.07374324671451},
#         'ICM': {"alpha": 41.72067133975204}
#     }
# )


In [ ]:
# SLIM -----------------------------------------    

# SLIM_recommender = MultiThreadSLIM_ElasticNet(
#         URM_train=ICM_combined.T,
#         verbose=False
#     )

# SLIM_recommender.fit(
#     alpha=0.00026894910579512645,
#     l1_ratio=0.08074126876487486,
#     topK=int(395.376118479588),
#     workers=6
# )

# SLIM_recommender.URM_train = URM_all

In [ ]:
# OTHER STUFF ----------------------------------

# rp3betaCBF_recommender= RP3betaCBFRecommender(
#     URM_train=URM_all,
#     ICM_train=ICM_all,
#     verbose=False
# )
#
# rp3betaCBF_recommender.fit(
#     topK=int(117.1),
#     alpha=0.9882,
#     beta=0.7703,
#     implicit=False
# )

# itemKNN_recommender= ItemKNNCFRecommender(
#     URM_train=URM_all,
#     verbose=False
# )
#
# itemKNN_recommender.fit(
#     topK=100,
#     shrink=50
# )
#
# pureSVD_recommender= PureSVDItemRecommender(
#     URM_train=URM_all,
#     verbose=False
# )
#
#
# pureSVD_recommender.fit(
#     num_factors=772,
#     topK= 599
# )

In [ ]:
# Generalized Merged Hybrid -----------------------------------------

# recommender = GeneralizedMergedHybridRecommender(
#     URM_train=URM_all,
#     recommenders=[
#         IALS_recommender,
#         rp3betaCombined_recommender
#         #SLIM_recommender
#     ],
#     verbose=False
# )

In [ ]:

# recommender.fit(
#     alphas=[
#         0.6836750866517823,
#         0.45969038157844144,
#         #0.2723405593515382
#     ]
# )

In [ ]:
# write_submission(recommender=recommender, target_users_path="input/data_target_users_test.csv",
#                     out_path='output/{}_submission.csv'.format(recommender.RECOMMENDER_NAME))